In [1]:
import os

In [2]:
%pwd

'd:\\Courses\\Krish_Naik\\Chicken_Disease_Classification\\research'

In [3]:
# move one folder up
os.chdir("../")

In [4]:
%pwd

'd:\\Courses\\Krish_Naik\\Chicken_Disease_Classification'

### 1. Update params.yaml with model parameters (DONE)

### 2. Update Entity

- Create a data class

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_dir: Path
    updated_base_model_dir: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class CallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    model_checkpoint_dir: Path

### 3. Update Configuration Manager 

- Create root directory
- Create a sub directory
- initialize the data class object

In [7]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self, 
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH
    ):
        # read base_model key in yaml file 
        self.config_file_path = read_yaml(config_file_path)
        # read params.yaml file
        self.params_file_path = read_yaml(params_file_path)
        
        # create directory -> "artifacts"; artifacts_root is present in config.yaml
        create_directories([self.config_file_path.artifacts_root])
    
    
    def get_callback_config(self) -> CallbacksConfig:
        
        callbacks_config = self.config_file_path.callbacks
        
        model_ckpt_dir = os.path.dirname(callbacks_config.model_checkpoint_dir)
        # create 2 sub directories in artifacts -> model_checkpoint_dir and tensorboard_root_log_dir
        create_directories([Path(model_ckpt_dir) , Path(callbacks_config.tensorboard_root_log_dir)])
        
        # create obj of BaseModelConfig class
        callbacks_config_obj = CallbacksConfig(
            root_dir = Path(callbacks_config.root_dir),
            tensorboard_root_log_dir = Path(callbacks_config.tensorboard_root_log_dir),
            model_checkpoint_dir = Path(callbacks_config.model_checkpoint_dir)
        )
        
        return callbacks_config_obj
    
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config_file_path.training
        base_model = self.config_file_path.base_model
        params = self.params_file_path
        training_data = os.path.join(self.config_file_path.data_ingestion.unzip_dir, "Chicken-fecal-images")
        
        # create root directory for training
        create_directories([Path(training.root_dir)])
        
        training_config_obj = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_dir = Path(training.trained_model_dir),
            updated_base_model_dir = Path(base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )
        
        return training_config_obj

### 4. Create Components

- callback component
- Training component

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [12]:
class Callbacks:
    
    def __init__(self, config: CallbacksConfig):
        self.config = config
    
    
    @property
    def _create_tensorboard_callbacks(self):
        """create timestamp folder save all the logs in that folder
        """
        timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
        tensorboard_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tensorborad_logs_at_{timestamp}")
        
        return tf.keras.callbacks.TensorBoard(log_dir = tensorboard_log_dir)
    
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.model_checkpoint_dir,
            save_best_only = True
        )
    
    def get_tensorboard_ckpt_callbacks(self):
        return [self._create_tensorboard_callbacks, self._create_ckpt_callbacks]
        

In [19]:
class Training:
    
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_dir)
    
    
    def train_valid_generator(self):
        
        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**data_generator_kwargs)
        
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **data_generator_kwargs
            )
        
        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )
        
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    
    
    def train(self, callbacks_lst: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callbacks_lst
        )
        
        
        self.save_model(path = self.config.trained_model_dir, model = self.model)


### 5 Create Pipeline

- For both callbacks and Training

In [20]:
try:
    config = ConfigurationManager()
    callbacks_config = config.get_callback_config()
    callbacks = Callbacks(config = callbacks_config)
    callback_list = callbacks.get_tensorboard_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training_obj = Training(config = training_config)
    training_obj.get_base_model()
    # Scale, train test split
    training_obj.train_valid_generator()
    # start training
    training_obj.train(callbacks_lst = callback_list)
    
    
except Exception as e:
    raise e

[2023-06-10 11:54:44,201: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-10 11:54:44,206: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-10 11:54:44,208: INFO: common: Created directory at: artifacts]
[2023-06-10 11:54:44,211: INFO: common: Created directory at: artifacts\callbacks\model_checkpoint_dir]
[2023-06-10 11:54:44,212: INFO: common: Created directory at: artifacts\callbacks\tensorboard_log_dir]
[2023-06-10 11:54:44,214: INFO: common: Created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/5
19/19 [==============================] - 73s 4s/step - loss: 12.8806 - accuracy: 0.5574 - val_loss: 4.7831 - val_accuracy: 0.6094
Epoch 2/5
19/19 [==============================] - 91s 5s/step - loss: 6.3563 - accuracy: 0.7432 - val_loss: 4.0672 - val_accuracy: 0.5312
Epoch 3/5
19/19 [==============================] - 82s 4s/step - loss: 4.7106 - accuracy: 0.7027 - val_